## Iteractively refine dataset labels

In [10]:
import os
import pathlib
import typing as t
import random
import requests
import json as json_package

import datasets
import tokenizers
import ipywidgets
import colorama

from config import *
import interactive_labeling


CN = colorama.Fore.RED
CT = colorama.Fore.YELLOW
CLN = colorama.Fore.CYAN
CSR = colorama.Style.RESET_ALL


VOCAB_SIZE = 6000
AUTOSAVE_IN_N_INSTANCES = 10


DATASET_SPLIT = "test"
CACHED_INDEX_FILENAME = "refined_indices.csv"
BRUTE_DATASET_DIR = "../data"
REFINED_DATASET_DIR = os.path.join(BRUTE_DATASET_DIR, "refined_datasets")

TARGET_DATASET_NAME = f"df_tokenized_split_0_120000_{VOCAB_SIZE}"

BRUTE_DATASET_URI = os.path.join(BRUTE_DATASET_DIR, TARGET_DATASET_NAME)
REFINED_DATASET_URI = os.path.join(REFINED_DATASET_DIR, TARGET_DATASET_NAME)
CACHED_INDEX_URI = os.path.join(REFINED_DATASET_DIR, CACHED_INDEX_FILENAME)


assert BRUTE_DATASET_URI != REFINED_DATASET_URI


random.seed(17)

In [11]:
tokenizer = tokenizers.Tokenizer.from_file(f"../tokenizers/{VOCAB_SIZE}_subwords/tokenizer.json")
tokenizer.get_vocab_size()

6000

## Load dataset

In [12]:
try:
    df = datasets.load_from_disk(REFINED_DATASET_URI)
    
    with open(CACHED_INDEX_URI, "r") as f_index:
        cached_indices = set(map(int, f_index.read().split(",")))
        
    print("Loaded pre-refined dataset from disk.")

except FileNotFoundError:
    df = datasets.load_from_disk(BRUTE_DATASET_URI)
    cached_indices = set()
    print("Loaded brute dataset from disk.")
    
df

Loaded brute dataset from disk.


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 114884
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
})

## Iteractive refinement

In [20]:
fn_rand = lambda: random.randint(0, df[DATASET_SPLIT].num_rows)
it_counter = 0

df_view_labels = df[DATASET_SPLIT]["labels"]
df_view_input_ids = df[DATASET_SPLIT]["input_ids"]

cls2id = {
    "no-op": 0,
    "seg": 1,
    "n-start": 2,
    "n-end": 3,
}

def print_labels(input_ids, labels, input_is_tokens: bool = False):
    seg_counter = 1
    
    print(end=CSR)
    print(end=f"{CLN}{seg_counter}.{CSR} ")
    
    if not input_is_tokens:
        tokens = list(map(tokenizer.id_to_token, input_ids))
    
    else:
        tokens = input_ids
    
    for i, (tok, lab) in enumerate(zip(tokens, labels)):
        if lab == cls2id["seg"]:
            seg_counter += 1
            print("\n\n", end=f"{CLN}{seg_counter}.{CSR} ")
        
        if lab == cls2id["n-start"]:
            print(end=CN)
            
        if lab == cls2id["n-end"]:
            print(end=CSR)
            
        print(tok, end=" ")
        
    return tokens

In [39]:
id_ = fn_rand()
while id_ in cached_indices:
    id_ = fn_rand()
    

input_ids = df_view_input_ids[id_]
labels = df_view_labels[id_]
tokens = print_labels(input_ids, labels)

1. [CLS] REQUERIMENTO DE INFORMAÇÃO Nº , DE 2020 ( Da Sra . Deputada Federal LU ##Í ##SA CA ##N ##Z ##IA ##NI ) 

2. Requer informações ao Ministro da Economia acerca do PL 38 ##87 / 2020 , de autoria do Poder Executivo , que prevê a remo ##ção do PIS e Co ##f ##ins do ro ##l de is ##enções do Programa Universidade para Todos ( Pro ##U ##ni ) . 

3. Senhor Presidente , Requeiro a Vossa Excelência , nos termos do artigo 50 , § 2º , da Constituição Federal , e nos artigos . 115 , I , e 116 do Regimento Interno da Câmara dos Deputados , a solicitação de informações ao Ministro da Economia para esclar ##ecimento aos dados que ense ##ja ##ram a remo ##ção da isenção do PIS e Co ##f ##ins para Institui ##ções de Ensino Superior Par ##tic ##ulares , com ou sem fins lucrativos , que ad ##er ##ir ##em ao Programa Universidade Para Todos ( Pro ##U ##ni ) , sem que seja inclu ##sa a Contribuição Social sobre Op ##era ##ções com B ##ens e Serviços – C ##B ##S , conforme disposto no 

4. Projeto de

In [40]:
interactive_labeling.open_example(tokens, labels)

In [41]:
ret = interactive_labeling.retrieve_refined_example()

it_counter += 1

if it_counter % AUTOSAVE_IN_N_INSTANCES == 0:
    df[DATASET_SPLIT]["labels"] = df_view_labels
    df[DATASET_SPLIT]["input_ids"] = df_view_input_ids
    
    # df.save_to_disk(dataset_dict_path=REFINED_DATASET_URI)
    
    print(f"Saved progress in '{REFINED_DATASET_URI}'.")
    
    it_counter = 0

In [42]:
print_labels(ret["tokens"], ret["labels"], input_is_tokens=True);

1. [CLS] REQUERIMENTO DE INFORMAÇÃO Nº , DE 2020 ( Da Sra . Deputada Federal LU ##Í ##SA CA ##N ##Z ##IA ##NI ) 

2. Requer informações ao Ministro da Economia acerca do PL 38 ##87 / 2020 , de autoria do Poder Executivo , que prevê a remo ##ção do PIS e Co ##f ##ins do ro ##l de is ##enções do Programa Universidade para Todos ( Pro ##U ##ni ) . 

3. Senhor Presidente , Requeiro a Vossa Excelência , nos termos do artigo 50 , § 2º , da Constituição Federal , e nos artigos . 115 , I , e 116 do Regimento Interno da Câmara dos Deputados , a solicitação de informações ao Ministro da Economia para esclar ##ecimento aos dados que ense ##ja ##ram a remo ##ção da isenção do PIS e Co ##f ##ins para Institui ##ções de Ensino Superior Par ##tic ##ulares , com ou sem fins lucrativos , que ad ##er ##ir ##em ao Programa Universidade Para Todos ( Pro ##U ##ni ) , sem que seja inclu ##sa a Contribuição Social sobre Op ##era ##ções com B ##ens e Serviços – C ##B ##S , conforme disposto no Projeto de Lei 